In [1]:
%%capture
!pip install -U "xarray[complete]" netCDF4 


In [8]:
import pandas as pd
import boto3
import xarray as xr
import os
import zarr
import s3fs
from dask.diagnostics import ProgressBar
from datetime import datetime
from yaml import safe_load
from numcodecs import blosc
import logging
logging.basicConfig(format='%(asctime)s %(message)s')

In [10]:
# Don't forget to disable fixed env
s3 = s3fs.S3FileSystem()
os.environ["DATALAKE_CONF_PATH"] = "/home/ec2-user/SageMaker/datalake/conf/data/configuration.yml"
confType = 'agera5ArgumentsSolarFlux'
ymlPath = os.environ["DATALAKE_CONF_PATH"]
conf_args = safe_load(open(ymlPath, 'r'))  # Use safe_load for security
confFile = conf_args[confType]
ageraS3 = s3fs.S3Map(root=confFile['s3Raw'], s3=s3, check=False)
logging.warning("Loading files to append")
ncToLoad = os.listdir(confFile['pathforReggrid'])

2024-05-06 14:23:17,023 Loading files to append


In [17]:
def convertRadiation(k):
    c = k/1000000
    return c

def netcdfChange(path
                 , file
                 ,variableName: str
                ):
    dt = file.split('_')[-2]
    dt = datetime.strptime(dt,'%Y%m%d')
    f = xr.open_dataset(path+file)
    f = f.drop_vars(['crs'])
    f = f.rename_vars(name_dict = {'Band1':variableName})
    #time_da = xr.DataArray({'time': dt})
    f = f.assign_coords(time = dt)
    f = f.expand_dims(dim="time")
    return f

In [18]:
for _nc in sorted(ncToLoad):
    varName = "Solar-Radiation-Flux"
    _file = confFile['pathforReggrid']+_nc
    logging.warning("Opening %s ", _nc)
    nc = netcdfChange(confFile['pathforReggrid'],_nc,varName)
    nc = nc.apply(convertRadiation)
    nc = nc[['time','lat','lon',varName]]
    logging.warning("Organizing dimensions")
    compressor = zarr.Blosc(cname='lz4', clevel= 1, shuffle=False)
    blosc.set_nthreads(8) 
    encoding = {vname: {'compressor': compressor,'chunks': (1,1,2000,7200)} for vname in nc.data_vars}
    with ProgressBar():
        #print("Hellooooo")
        nc.to_zarr(ageraS3,  mode='a', append_dim='time', consolidated=True)
    logging.warning("Insert finished")
logging.warning("Process completed")

2024-05-06 14:35:20,380 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240101_final-v1.1.nc 
2024-05-06 14:35:20,578 Organizing dimensions
2024-05-06 14:35:20,579 Insert finished
2024-05-06 14:35:20,580 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240102_final-v1.1.nc 
2024-05-06 14:35:20,772 Organizing dimensions
2024-05-06 14:35:20,773 Insert finished
2024-05-06 14:35:20,774 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240103_final-v1.1.nc 


Hellooooo
Hellooooo


2024-05-06 14:35:20,965 Organizing dimensions
2024-05-06 14:35:20,966 Insert finished
2024-05-06 14:35:20,967 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240104_final-v1.1.nc 
2024-05-06 14:35:21,158 Organizing dimensions
2024-05-06 14:35:21,159 Insert finished
2024-05-06 14:35:21,160 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240105_final-v1.1.nc 


Hellooooo
Hellooooo


2024-05-06 14:35:21,350 Organizing dimensions
2024-05-06 14:35:21,352 Insert finished
2024-05-06 14:35:21,352 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240106_final-v1.1.nc 
2024-05-06 14:35:21,543 Organizing dimensions
2024-05-06 14:35:21,544 Insert finished
2024-05-06 14:35:21,545 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240107_final-v1.1.nc 


...

Hellooooo
Hellooooo


2024-05-06 14:35:25,962 Organizing dimensions
2024-05-06 14:35:25,963 Insert finished
2024-05-06 14:35:25,964 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240130_final-v1.1.nc 
2024-05-06 14:35:26,155 Organizing dimensions
2024-05-06 14:35:26,156 Insert finished
2024-05-06 14:35:26,156 Opening Solar-Radiation-Flux_C3S-glob-agric_AgERA5_20240131_final-v1.1.nc 


Hellooooo
Hellooooo


2024-05-06 14:35:26,347 Organizing dimensions
2024-05-06 14:35:26,348 Insert finished
2024-05-06 14:35:26,348 Process completed


Hellooooo


In [19]:
nc

<xarray.Dataset> Size: 115MB
Dimensions:               (time: 1, lat: 2000, lon: 7200)
Coordinates:
  * time                  (time) datetime64[ns] 8B 2024-01-31
  * lat                   (lat) float64 16kB -49.98 -49.93 ... 49.92 49.97
  * lon                   (lon) float64 58kB -180.0 -179.9 ... 179.9 180.0
Data variables:
    Solar-Radiation-Flux  (time, lat, lon) float64 115MB nan nan nan ... nan nan